In [1]:
from rpy2.robjects.vectors import StrVector
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import r, pandas2ri
import pandas as pd
import numpy as np
import scipy.stats as sstats

from collections import namedtuple
class ProteinSet(object):
    def __init__(self, proteindict, database):
        """
        Function: 
        Making a global variable of the variables that are gonna be used through the whole class. 
        
        Variables: 
        self.proteindict = a dictionary with a term and a list of proteins per item. 
        self.database = the name of the database. 
        """
        self.proteindict = { name : set(p) for name, p in proteindict.items() }
        self.database = database
    
    def enrich(self, otherset, background):
        """
        Function: 
        This function takes 3 sets of proteins (or genes) and uses them to make an enrichment using either fisher's exact test or 
        the chi2 (depending on how big the sets are). 
        
        Variables: 
        list_res = a list with lists that will later be turned into a dataframe. Eech list within the list will have information
        about a row in the table. 
        name = the name of the drug 
        pset = a set of proteins that are targets of the drug. 
        term = the name in a list. 
        proteins = the proteins in a list. 
        results = the enrichment results in a NamedTuple. 
        l_results = the enrichment results turned into a list. 
        joined = the name of the database and the term merged with the l_results list. 
        df_final = a dataframe with all the enrichment results. 
        """
        list_res = []
        for name, pset in self.proteindict.items():
            term = [self.database] + [name] 
            proteins = list(pset)
            results = self.set_enrichment(pset, otherset, background)
            l_results = list(results)
            joined = term + l_results
            joined.append(proteins)
            list_res.append(joined)
        
        df_final = pd.DataFrame(list_res)
        df_final.columns = ['Database', 'Name', 'oddsratio', 'c2statistic', 'pvalue', 'table', 'method', 'proteins'] 
        return df_final
    
    def set_enrichment(self, your_set, other_set, universe, abcd_values=False):
    
        resTuple = namedtuple("setEnrichmentResult", [ 'oddsratio', 'c2statistic', 'pvalue', 'table', 'method'])

        universe  = set(universe)
        your_set  = set(your_set) & universe
        other_set = set(other_set) & universe

        a = your_set & other_set
        b = other_set - your_set
        c = your_set - other_set
        d = universe - (your_set | other_set)

        table = [ [len(a), len(b)], [len(c), len(d)]]
        if min(min(table)) <= 5:
            method = 'fisher'
            oddsratio, p = sstats.fisher_exact(table)
            chi2 = None
        else:
            method = 'chi2'
            chi2, p, dof, expected = sstats.chi2_contingency(table)
            oddsratio = 100
            if table[1][0] > 0 and table[0][1] > 0:
                oddsratio = table[0][0] * table[1][1] / (table[1][0] * table[0][1])
            else:
                oddsratio = np.inf

        if abcd_values:
            return resTuple(oddsratio, chi2, p, [[a,b],[c,d]], method)
        else:
            return resTuple(oddsratio, chi2, p, table, method)
    
def protein_to_entrez(other_set1): 
    """
    Function: 
    This function takes a dataset containing ensembl protein id's and turns them into entrez gene id's. 
    
    Variables: 
    biomart_data = a dataset with entrez gene id's and their corresponding ensembl protein id's. 
    other_set1 = a list of entrez gene id's. 
    get_ens = merged dataset with other_set1 and biomart_data. 
    get_ens_filtered = the same dataset as get_ens but without some of the columns that are not important. 
    """
    biomart_data = pd.read_csv("biomart.tsv", 
                  sep='\t', 
                  names=["gene", "transcript", "protein", "Entrez", "Uniprot", "name"])

    get_ens = pd.merge(other_set1, biomart_data, on=["protein"]) 
    get_ens = get_ens.dropna(subset=['Entrez'])
    get_ens['Entrez'] = get_ens['Entrez'].astype(int)
    get_ens_filtered = get_ens.drop(["gene", "transcript", "Uniprot"], axis=1)
    
    return get_ens_filtered

def make_dictio_DT(): 
    """
    Function: 
    This function makes a dictionary with a drug and a list of proteins that are targets of that drug.
    
    Variable: 
    mapped = dataset with drugs and their targets. 
    dictio = a dictionary with a drug and the corresponding targets (proteins).
    """
    mapped = pd.read_csv('mapped_DB_STITCH.tsv', sep='\t')
    mapped['protein'] = mapped['protein'].map(lambda x: x.lstrip('9606.'))
    mapped = mapped[['CID', 'InChIKey', 'DrugBank ID', 'Name', 'protein', 'combined_score']].drop_duplicates()
    
    get_entr_filtered = protein_to_entrez(mapped)
    
    dictio = {}
    for i in get_entr_filtered['Name'].unique(): 
        dictio[i] = [get_entr_filtered['Entrez'][j] for j in get_entr_filtered[get_entr_filtered['Name']==i].index]
    
    return dictio 
    
def cluster_profiler_KEGG(clusterProfiler, data): 
    """
    Function: 
    This function uses the R-package: clusterProfiler to enrich a list of genes for KEGG pathways. You will get back a list
    of KEGG pathways and the sets of genes part of that pathway. The list is sorted by the adjusted p-value.
    
    Variables: 
    enrich_KEGG = the enrichment for the KEGG pathways. This variable is in RS4 format. 
    KEGGdat = the results of the enrichment in a normal (r) dataframe format.
    df = to get the KEGG enrichment results into a pandas dataframe format.
    """
    
    enrich_KEGG = clusterProfiler.enrichKEGG(data, organism = 'hsa', keyType = 'kegg', pvalueCutoff = 0.05, pAdjustMethod = 'BY')
    KEGGdat = enrich_KEGG.slots['result']
    
    df = pd.DataFrame(index=range(len(KEGGdat[0])))
    df['ID'] = KEGGdat[0]
    df['Description'] = KEGGdat[1]
    df['GeneRatio'] = KEGGdat[2]
    df['pvalue'] = KEGGdat[4]
    df['padjust'] = KEGGdat[5]
    df['Count'] = KEGGdat[8]
    empty_lijst_KEGG = []
    for x in KEGGdat[7]:
        per_pathway = []
        lijst = x.split('/')
        eg_KEGG = clusterProfiler.bitr(lijst, fromType="ENTREZID", toType="ENSEMBLPROT", OrgDb="org.Hs.eg.db")
        per_pathway += eg_KEGG[1]
        empty_lijst_KEGG.append(per_pathway)
    df['ProteinID'] = empty_lijst_KEGG
    df = df[df['padjust']<0.05]
    
    return df

def cluster_profiler_GO_MF(clusterProfiler, data): 
    """
    Function: 
    This function uses the R-package: clusterProfiler to enrich a list of genes for GO annotations (specifically molecular
    function). You will get back a list of GO annotations with the sets of genes that have something to do with that GO anno-
    tation. This list is sorted by the adjusted p-value. 
    
    Variables: 
    enrich_GO_MF = the enrichment for the GO annotations (molecular function). This variable is in RS4 format. 
    MFdat = the results of the enrichment in a normal (r) dataframe format. 
    df_GO_MF = to get the GO annotation enrichment results into a pandas dataframe format. 
    """
    
    enrich_GO_MF = clusterProfiler.enrichGO(data, 'org.Hs.eg.db', ont = 'MF', pvalueCutoff = 0.05, pAdjustMethod = 'BY')
    MFdat = enrich_GO_MF.slots['result']
    
    df_GO_MF = pd.DataFrame(index=range(len(MFdat[0])))
    df_GO_MF['ID'] = MFdat[0]
    df_GO_MF['Description'] = MFdat[1]
    df_GO_MF['GeneRatio'] = MFdat[2]
    df_GO_MF['pvalue'] = MFdat[4]
    df_GO_MF['padjust'] = MFdat[5]
    df_GO_MF['Count'] = MFdat[8]
    empty_lijst_GO_MF = []
    for x in MFdat[7]: 
        lijst = x.split('/')
        empty_lijst_GO_MF.append(lijst)
    df_GO_MF['GeneID'] = empty_lijst_GO_MF
    df_GO_MF = df_GO_MF[df_GO_MF['padjust']<0.05]
    
    return df_GO_MF

def cluster_profiler_GO_CC(clusterProfiler, data): 
    """
    Function: 
    This function uses the R-package: clusterProfiler to enrich a list of genes for GO annotations (specifically cellular com-
    ponent). You will get back a list of GO annotations with the sets of genes that have something to do with that GO anno-
    tation. This list is sorted by the adjusted p-value. 
    
    Variables: 
    enrich_GO_CC = the enrichment for the GO annotations (cellular component). This variable is in RS4 format. 
    CCdat = the results of the enrichment in a normal (r) dataframe format. 
    df_GO_CC = to get the GO annotation enrichment results into a pandas dataframe format.
    """
    
    enrich_GO_CC = clusterProfiler.enrichGO(data, 'org.Hs.eg.db', ont = 'CC', pvalueCutoff = 0.05, pAdjustMethod = 'BY')
    CCdat = enrich_GO_CC.slots['result']
    
    df_GO_CC = pd.DataFrame(index=range(len(CCdat[0])))
    df_GO_CC['ID'] = CCdat[0]
    df_GO_CC['Description'] = CCdat[1]
    df_GO_CC['GeneRatio'] = CCdat[2]
    df_GO_CC['pvalue'] = CCdat[4]
    df_GO_CC['padjust'] = CCdat[5]
    df_GO_CC['Count'] = CCdat[8]
    empty_lijst_GO_CC = []
    for x in CCdat[7]: 
        lijst = x.split('/')
        lijst = list(map(int, lijst))
        empty_lijst_GO_CC.append(lijst)
    df_GO_CC['GeneID'] = empty_lijst_GO_CC
    df_GO_CC = df_GO_CC[df_GO_CC['padjust']<0.05]
    
    return df_GO_CC

def cluster_profiler_GO_BP(clusterProfiler, data):
    """
    Function: 
    This function uses the R-package: clusterProfiler to enrich a list of genes for GO annotations (specifically biological
    process). You will get back a list of GO annotations with the sets of genes that have something to do with that GO anno-
    tation. This list is sorted by the adjusted p-value.
    
    Variables: 
    enrich_GO_BP = the enrichment for the GO annotations (biological process). This variable is in RS4 format. 
    BPdat = the results of the enrichment in a normal (r) dataframe format.
    df_GO_BP = to get the GO annotation enrichment results into a pandas dataframe format.
    """
    
    enrich_GO_BP = clusterProfiler.enrichGO(data, 'org.Hs.eg.db', ont = 'BP', pvalueCutoff = 0.05, pAdjustMethod = 'BY')
    BPdat = enrich_GO_BP.slots['result']
    
    df_GO_BP = pd.DataFrame(index=range(len(BPdat[0])))
    df_GO_BP['ID'] = BPdat[0]
    df_GO_BP['Description'] = BPdat[1]
    df_GO_BP['GeneRatio'] = BPdat[2]
    df_GO_BP['pvalue'] = BPdat[4]
    df_GO_BP['padjust'] = BPdat[5]
    df_GO_BP['Count'] = BPdat[8]
    
    empty_lijst_GO_BP = []
    for x in BPdat[7]: 
        lijst = x.split('/')
        lijst = list(map(int, lijst))
        empty_lijst_GO_BP.append(lijst)
        
    df_GO_BP['GeneID'] = empty_lijst_GO_BP
    df_GO_BP = df_GO_BP[df_GO_BP['padjust']<0.05]  
    
    return df_GO_BP

def Reactome(ReactomePA, data): 
    """
    Function: 
    This function uses the R-package: ReactomePA to enrich a list of genes for Reactome pathways. You will get back a list of 
    Reactome pathways with the sets of genes that have something to do with that Reactome pathway. This list will be sorted 
    by the adjusted p-value. 
    
    Variable: 
    enrich_Reactome = the enrichment for the reactome pathways. This variable is in RS4 format. 
    ReactomeDat = the results of the enrichment in a normal (r) dataframe format. 
    df_Reactome = to get the Reactome enrichment results into a pandas dataframe format. 
    """
    
    enrich_Reactome = ReactomePA.enrichPathway(gene=data, pvalueCutoff = 0.05, readable = True, pAdjustMethod = 'BY', organism = "human")
    ReactomeDat = enrich_Reactome.slots['result']
    
    df_Reactome = pd.DataFrame(index=range(len(ReactomeDat[0])))
    df_Reactome['ID'] = ReactomeDat[0]
    df_Reactome['Description'] = ReactomeDat[1]
    df_Reactome['GeneRatio'] = ReactomeDat[2]
    df_Reactome['pvalue'] = ReactomeDat[4]
    df_Reactome['padjust'] = ReactomeDat[5]
    df_Reactome['Count'] = ReactomeDat[8]
    
    entrez_list = []
    for x in ReactomeDat[7]:
        input_bitr = x.split('/')
        eg = clusterProfiler.bitr(input_bitr, fromType="SYMBOL", toType="ENTREZID", OrgDb="org.Hs.eg.db")
        entrez_list.append(eg[1]) 

    df_Reactome['GeneID'] = entrez_list
    df_Reactome = df_Reactome[df_Reactome['padjust']<0.05]
    
    return df_Reactome

def main_enrichments_ACR():
    """
    Function: 
    This function calls all the other functions that have something to do with enriching with R. It also imports the needed
    R-packages. Before this function can be called please make sure you have the packages already installed and the packages 
    are in the right folder (/Users/user/Anaconda3/Lib/R/library). 
    
    Variables: 
    data = the list of genes read in with a R-code. 
    clusterProfiler = the package: clusterProfiler imported using rpy2.
    ReactomePA = the package: ReactomePA imported using rpy2
    """
    data = robjects.r("scan('/Users/myrthedehaan/Downloads/Internship_LUMC/test_list_genes1.0.txt', what='', sep='\n', skip = 1)")
    ensembl = pd.read_csv('STITCH_proteins.txt')
    
    get_entr_filtered_ens = protein_to_entrez(ensembl)
    
    d = {'print.me': 'print_dot_me', 'print_me': 'print_uscore_me'}
    
    clusterProfiler = rpackages.importr('clusterProfiler')
    GO_CC_en_res = cluster_profiler_GO_CC(clusterProfiler, data)
    
    dictionary = {} 
    for index, row in GO_CC_en_res.iterrows():
        dictionary[row['ID']] = row['GeneID']
        
    #KEGG_en_res = cluster_profiler_KEGG(clusterProfiler, data)
    dictio = make_dictio_DT()
    database = ['KEGG', 'GO_MF', 'GO_CC', 'GO_BP', 'Reactome']
    enrichment_call = ProteinSet(dictionary, database[2])
    #dfObj = pd.DataFrame()
    super_x = []
    for drugs, targets in dictio.items():
        df = enrichment_call.enrich(targets, get_entr_filtered_ens['Entrez'])
        df['drug'] = drugs
        super_x.append(df)
    dfObj = pd.concat(super_x) 
    print(dfObj)
    #print(dfObj.groupby('drug').transform(min)) 
    
    #GO_MF_en_res = cluster_profiler_GO_MF(clusterProfiler, data)
    #GO_CC_en_res = cluster_profiler_GO_CC(clusterProfiler, data)
    #GO_BP_en_res = cluster_profiler_GO_BP(clusterProfiler, data)
    #reactome_en_res = Reactome(ReactomePA, data)
    

main_enrichments_ACR()   

R[write to console]: Read 1216 items

R[write to console]: Error: package or namespace load failed for ‘clusterProfiler’ in library.dynam(lib, package, package.lib):
 shared object ‘rlang.dylib’ not found

R[write to console]: Error in library.dynam(lib, package, package.lib) : 
  shared object ‘rlang.dylib’ not found
Calls: <Anonymous> ... loadNamespace -> namespaceImport -> loadNamespace -> library.dynam



RRuntimeError: Error in library.dynam(lib, package, package.lib) : 
  shared object ‘rlang.dylib’ not found
Calls: <Anonymous> ... loadNamespace -> namespaceImport -> loadNamespace -> library.dynam


In [1]:
import rpy2.robjects.packages as rpackages
ReactomePA = rpackages.importr('clusterProfiler')

//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error: package or namespace load failed for ‘clusterProfiler’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib':
  dlopen(/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib, 6): Library not loaded: @rpath/R/lib/libRlapack.dylib
  Referenced from: /anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib
  Reason: image not found

  warnings.warn(x, RRuntimeWarning)
//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Error in dyn.load(file, DLLpath = DLLpath, ...) : 
  unable to load shared object '/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib':
  dlopen(/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib, 6): Library not loaded: @rpath/R/lib/libRlap

RRuntimeError: Error in dyn.load(file, DLLpath = DLLpath, ...) : 
  unable to load shared object '/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib':
  dlopen(/anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib, 6): Library not loaded: @rpath/R/lib/libRlapack.dylib
  Referenced from: /anaconda3/envs/Internship_LUMC/lib/R/library/Matrix/libs/Matrix.dylib
  Reason: image not found


In [4]:
import rpy2.robjects as robjects
data = robjects.r("BiocManager::install('GO.db')")


//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package(s) 'GO.db'

  warnings.warn(x, RRuntimeWarning)
//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://bioconductor.org/packages/3.9/data/annotation/src/contrib/GO.db_3.8.2.tar.gz'

  warnings.warn(x, RRuntimeWarning)
//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning:  length 31820866 bytes (30.3 MB)

  warnings.warn(x, RRuntimeWarning)
//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: downloaded 30.3 MB


  warnings.warn(x, RRuntimeWarning)
//anaconda3/envs/Internship_LUMC/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Old packages: 'BiocParallel', 'fgsea', 'RcppArmadillo'

  warnings.warn(x, RRuntimeWarning)


Update all/some/none? [a/s/n]: 
a
